In [1]:
from sklearn.model_selection import train_test_split
from __future__ import print_function
import numpy as np                 # to use numpy arrays
import tensorflow as tf            # to specify and run computation graphs
import tensorflow_datasets as tfds # to load training data
import keras
import pandas as pd
from keras import backend
from tensorflow.keras import Model, initializers, regularizers, constraints
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt 

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-fekmakat because the default path (/home/cse479/maxnguyen/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
inputs = pd.read_csv('input.csv')


In [3]:
print(inputs)

    Country Code ContinentCode    Year  \
0    Canada  CAN           NaN  2005.0   
1    Canada  CAN           NaN  2006.0   
2    Canada  CAN           NaN  2007.0   
3    Canada  CAN           NaN  2008.0   
4    Canada  CAN           NaN  2009.0   
..      ...  ...           ...     ...   
115     USA  USA           NaN  2015.0   
116     USA  USA           NaN  2016.0   
117     USA  USA           NaN  2017.0   
118     USA  USA           NaN  2018.0   
119     NaN  NaN           NaN     NaN   

     Cost of starting a business % of income per capita  \
0                                                  0.9    
1                                                  0.8    
2                                                  0.8    
3                                                  0.4    
4                                                  0.4    
..                                                 ...    
115                                                1.1    
116                    

In [4]:
keys = []
for it in inputs:
    keys.append(it)

In [5]:
print(keys)

['Country', 'Code', 'ContinentCode', 'Year', 'Cost of starting a business % of income per capita', 'Real interest rate: Bank lending rate minus inflation', 'Stock market capitalization w/o top 10 firms percent of total market cap', 'Ratio of car sales to population', 'Ratio of commercial vehicles sales to population', 'Capital investment as percent of GDP', 'Labor force participation rate', 'Fixed broadband internet subscribers per 100 people', 'Research and development expenditure percent of GDP', 'Property rights index (0-100)', 'Freedom from corruption index (0-100)', 'Fiscal freedom index (0-100)', 'Business freedom index (0-100)', 'Labor freedom index (0-100)', 'Monetary freedom index (0-100)', 'Trade freedom index (0-100)', 'Investment freedom index (0-100)', 'Financial freedom index (0-100)', 'Economic freedom overall index (0-100)', 'Population ages 65 and above percent of total', 'Public spending on education percent of public spending', 'Student teacher ratio primary school',

In [6]:
distribution = [4, 9, 24, 26, 27, 28] #horrible design

print(keys[distribution[4] - 1])

Savings as percent of GDP


In [7]:
number_of_tests = len(inputs[keys[0]])

In [20]:
input_short_term = []
input_long_term = []
output = []
for i in range(number_of_tests):
    instance = []
    if pd.isna(inputs[keys[0]][i]):
        continue
    for j in range(distribution[0], distribution[1]):         
        instance.append(round(float(inputs[keys[j]][i])/100, 2))
        
    input_short_term.append(instance)
    instance = []
    for j in range(distribution[1], distribution[2]):
        instance.append(round(float(inputs[keys[j]][i])/100, 2))
    for j in range(distribution[3], distribution[4]):
        instance.append(round(float(inputs[keys[j]][i])/100, 2))
    input_long_term.append(instance)
    
    output.append(round(float(inputs[keys[distribution[5] - 1]][i]), 2))

# print(training_input_short_term)
# print(training_input_long_term)
# print(output)

In [21]:
input_short_term = np.array(input_short_term)
input_long_term = np.array(input_long_term)
output = np.array(output)

In [11]:
actual_number_of_tests = len(input_short_term)

In [ ]:
model = Sequential()
model.add(Dense(50, activation='relu', input_shape = input_short_term[0].shape))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='mean_absolute_error')

In [ ]:
#Fitting the model
history = model.fit(training_input_short_term, output, epochs = 10, batch_size = 32, verbose = 1)

In [ ]:
model = Sequential()
model.add(Dense(50, activation='relu', input_shape = input_long_term[0].shape))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='mean_absolute_error')

In [ ]:
#Fitting the model
history = model.fit(training_input_long_term, output, epochs = 10, batch_size = 32, verbose = 1)

In [12]:
short_term_input1 = Input(shape=(5,))
short_term_input2 = Input(shape=(5,))
long_term_input = Input(shape=(16,))
dense7 = Dense(50, activation='relu')
x1 = dense7(short_term_input1)
x1 = Dropout(0.5)(x1)
x1 = Dense(50)(x1)
x1 = LeakyReLU(0.2)(x1)

con1 = Concatenate(axis=1)([x1, short_term_input2, long_term_input])
#con1 = LSTM(9, return_sequences = True)(con1)
con1 = Dense(50, activation='relu')(con1)

#x2 = LSTM(3, return_sequences = True)(long_term_input)

x2 = Dense(50, activation='relu')(long_term_input)
con2 = Concatenate(axis=1)([con1,x2] ) 
output = Dense(1, activation=None)(con2)
model = Model(inputs=[short_term_input1, short_term_input2, long_term_input], outputs=output)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 5)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 50)           300         input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 50)           0           dense[0][0]                      
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 50)           2550        dropout[0][0]                    
_______________________________________________________________________________________

In [13]:
model.compile(optimizer='adam', loss='mean_absolute_error')

In [14]:
es = EarlyStopping(monitor = 'loss', mode = 'min', verbose = 1, patience = 5)

In [ ]:
#[training_input_short_term, training_input_short_term, training_input_long_term], train_output, [testing_input_short_term, testing_input_short_term, testing_input_long_term], test_output = train_test_split([input_short_term, input_short_term, input_long_term], output, shuffle=True)

In [23]:
#Fitting the model
history = model.fit([input_short_term, input_short_term, input_long_term], output, epochs = 1000, batch_size = 32, verbose = 1, shuffle=True, callbacks=[es])

Epoch 1/1000
4/4 [==============================] - 0s 2ms/step - loss: 1.8122
Epoch 2/1000
4/4 [==============================] - 0s 3ms/step - loss: 1.8117
Epoch 3/1000
4/4 [==============================] - 0s 3ms/step - loss: 1.8112
Epoch 4/1000
4/4 [==============================] - 0s 3ms/step - loss: 1.8107
Epoch 5/1000
4/4 [==============================] - 0s 2ms/step - loss: 1.8102
Epoch 6/1000
4/4 [==============================] - 0s 2ms/step - loss: 1.8097
Epoch 7/1000
4/4 [==============================] - 0s 2ms/step - loss: 1.8092
Epoch 8/1000
4/4 [==============================] - 0s 3ms/step - loss: 1.8086
Epoch 9/1000
4/4 [==============================] - 0s 2ms/step - loss: 1.8083
Epoch 10/1000
4/4 [==============================] - 0s 2ms/step - loss: 1.8077
Epoch 11/1000
4/4 [==============================] - 0s 2ms/step - loss: 1.8072
Epoch 12/1000
4/4 [==============================] - 0s 2ms/step - loss: 1.8068
Epoch 13/1000
4/4 [==============================